In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    462 non-null    object 
 1   Address                 462 non-null    object 
 2   Contact                 438 non-null    object 
 3   Map Link                462 non-null    object 
 4   rating                  462 non-null    float64
 5   No of reviews           462 non-null    object 
 6   Social media / website  368 non-null    object 
 7   Menu Link               92 non-null     object 
 8   Restaurant Keyword      353 non-null    object 
 9   Restaurant Speciality   209 non-null    object 
dtypes: float64(1), object(9)
memory usage: 36.2+ KB


In [3]:
df["Restaurant Keyword"].head(60)

0                          Bar/beer
1     Traditional Moldavian cuisine
2                     Mixed kitchen
3                  European cuisine
4                   Italian Cuisine
5                     Mixed kitchen
6                     Mixed kitchen
7                     Mixed kitchen
8                     Mixed kitchen
9                     Mixed kitchen
10                    Mixed kitchen
11                    Mixed kitchen
12                    Mixed kitchen
13                    Mixed kitchen
14                    Mixed kitchen
15                    Mixed kitchen
16                    Mixed kitchen
17                    Mixed kitchen
18                    Mixed kitchen
19                    Mixed kitchen
20                    Mixed kitchen
21                    Mixed kitchen
22                    Mixed kitchen
23                    Mixed kitchen
24                    Mixed kitchen
25                    Mixed kitchen
26                    Mixed kitchen
27                    Mixed 

In [4]:
df['Restaurant Speciality'] = df['Restaurant Speciality'].fillna('')

# Split the values by comma and strip any extra whitespace
all_specialities = df['Restaurant Speciality'].str.split(',').explode().str.strip()

# Get unique values and sort them
unique_specialities = all_specialities.unique()
unique_specialities.sort()

# Display the unique values
print(unique_specialities)

['' 'American' 'And European' 'Arabic' 'Armenian' 'Asian' 'Azerbaijani'
 'Azerbaijani cuisine' 'Bakery products' 'Baking' 'Bar' 'Barbecue'
 'Breakfast' 'British' 'Burgers' 'Cafe' 'Cafeteria' 'Caucasian'
 'Central European' 'Chinese' 'Cocktails' 'Crayfish' 'Czech'
 'Czech cuisine' 'Desserts' 'Dishes of Chinese' 'East European'
 'East European cuisine' 'European' 'FUUZHN' 'Fast food' 'French'
 'French cuisine' 'Fusion' 'Fuzhn' 'Greek' 'Greek tavern' 'Grill'
 'Healthy food' 'Indian' 'International' 'Israeli' 'Italian' 'Japanese'
 'Kebab' 'Khinkali' 'Latin American' 'Mediterranean' 'Mexican'
 'Middle Eastern' 'Moldavian' 'Nepalskaya' 'Pasta' 'Pins' 'Pizza'
 'Pub with a brewery' 'Ramen' 'Risotto' 'Romanian' 'Romanian cuisine'
 'Russian' 'Scandinavian' 'Seafood' 'Shaverma' 'Spanish' 'Steak house'
 'Steak-house' 'Steaks' 'Thai' 'Tom Yam' 'Traditional Moldavian' 'Turkish'
 'Ukrainian' 'Vietnamese' 'Wok' 'and Vietnamese cuisine' 'baking' 'bar'
 'barbecue' 'bars with restaurant' 'beer' 'beer res

In [5]:
df.columns

Index(['Name', 'Address', 'Contact', 'Map Link', 'rating', 'No of reviews',
       'Social media / website', 'Menu Link', 'Restaurant Keyword',
       'Restaurant Speciality'],
      dtype='object')

In [6]:
df['Restaurant Keyword'] = df['Restaurant Keyword'].fillna('')

# Split the values by comma and strip any extra whitespace
all_keyword = df['Restaurant Keyword'].str.split(',').explode().str.strip()
#print(all_keyword)
# Get unique values and sort them
unique_keyword = all_keyword.unique()
unique_keyword.sort()

# Display the unique values
print(unique_keyword)

['' 'American cuisine' 'Armenian cuisine' 'Asian cuisine' 'Bar/beer'
 'European cuisine' 'Fast Food' 'Georgian cuisine' 'Indian food'
 'Italian Cuisine' 'Mexican cuisine' 'Mixed kitchen' 'Russian kitchen'
 'Seafood' 'Traditional Moldavian cuisine' 'coffee house']


In [7]:
import sqlite3

def create_database():
    # Connect to SQLite database (creates the file if it doesn't exist)
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()

    schema = """
    CREATE TABLE IF NOT EXISTS Keywords (
        keyword_id INTEGER PRIMARY KEY AUTOINCREMENT,
        keyword_name TEXT
    );
    CREATE TABLE IF NOT EXISTS Speciality (
        speciality_id INTEGER PRIMARY KEY AUTOINCREMENT,
        speciality_name TEXT
    );
    CREATE TABLE IF NOT EXISTS Restaurant_Info (
        restaurant_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        map_link TEXT,
        contact_info TEXT,
        rating FLOAT,
        no_of_reviews INT,
        address TEXT,
        menu_link TEXT
    );
    CREATE TABLE IF NOT EXISTS Restaurant_Keywords (
        restaurant_id INTEGER,
        keyword_id INTEGER,
        PRIMARY KEY (restaurant_id, keyword_id),
        FOREIGN KEY (restaurant_id) REFERENCES Restaurant_Info(restaurant_id),
        FOREIGN KEY (keyword_id) REFERENCES Keywords(keyword_id)
    );
    CREATE TABLE IF NOT EXISTS Restaurant_Speciality (
        restaurant_id INTEGER,
        speciality_id INTEGER,
        PRIMARY KEY (restaurant_id, speciality_id),
        FOREIGN KEY (restaurant_id) REFERENCES Restaurant_Info(restaurant_id),
        FOREIGN KEY (speciality_id) REFERENCES Speciality(speciality_id)
    );
    CREATE TABLE IF NOT EXISTS Restaurant_Social_Media (
        restaurant_id INTEGER,
        url TEXT,
        PRIMARY KEY (restaurant_id, url),
        FOREIGN KEY (restaurant_id) REFERENCES Restaurant_Info(restaurant_id)
    );
    """


    # Execute the schema
    cursor.executescript(schema)

    # Commit and close connection
    conn.commit()
    conn.close()


create_database()

In [8]:
import sqlite3

def populate_speciality_table():
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()

    # Insert into Speciality table
    for speciality in unique_specialities:
        if speciality:  # Avoid empty strings
            cursor.execute("INSERT INTO Speciality (name) VALUES (?)", (speciality,))

    conn.commit()
    conn.close()

populate_speciality_table()

In [9]:
def populate_keywords_table():
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()


    # Insert into Keywords table
    for keyword in unique_keyword:
        if keyword:  # Avoid empty strings
            cursor.execute("INSERT INTO Keywords (keyword_name) VALUES (?)", (keyword,))

    conn.commit()
    conn.close()

populate_keywords_table()


In [10]:
def populate_restaurant_info_table(df):
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()

    # Insert into Restaurant_Info table
    for index, row in df.iterrows():
        cursor.execute("""
            INSERT INTO Restaurant_Info (name, map_link, contact_info, rating, no_of_reviews, address, menu_link)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (row['Name'], row['Map Link'], row['Contact'], row['rating'], row['No of reviews'], row['Address'], row['Menu Link']))

    conn.commit()
    conn.close()

populate_restaurant_info_table(df)


In [11]:
def populate_restaurant_keywords_table(df):
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()

    for index, row in df.iterrows():
        keywords = row['Restaurant Keyword'].split(',')
        keywords = [kw.strip() for kw in keywords if kw.strip()]

        for keyword in keywords:
            # Get restaurant_id
            cursor.execute("SELECT restaurant_id FROM Restaurant_Info WHERE name=?", (row['Name'],))
            restaurant_id = cursor.fetchone()[0]

            # Get keyword_id
            cursor.execute("SELECT keyword_id FROM Keywords WHERE keyword_name=?", (keyword,))
            keyword_id = cursor.fetchone()[0]

            # Check if the (restaurant_id, keyword_id) combination already exists
            cursor.execute("""
                SELECT 1 FROM Restaurant_Keywords
                WHERE restaurant_id=? AND keyword_id=?
            """, (restaurant_id, keyword_id))
            exists = cursor.fetchone()

            if not exists:
                # Insert into Restaurant_Keywords table
                cursor.execute("""
                    INSERT INTO Restaurant_Keywords (restaurant_id, keyword_id)
                    VALUES (?, ?)
                """, (restaurant_id, keyword_id))

    conn.commit()
    conn.close()

populate_restaurant_keywords_table(df)


In [12]:
def populate_restaurant_speciality_table(df):
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()

    for index, row in df.iterrows():
        specialities = row['Restaurant Speciality'].split(',')
        specialities = [sp.strip() for sp in specialities if sp.strip()]

        for speciality in specialities:
            # Get restaurant_id
            cursor.execute("SELECT restaurant_id FROM Restaurant_Info WHERE name=?", (row['Name'],))
            restaurant_id = cursor.fetchone()[0]

            # Get speciality_id
            cursor.execute("SELECT speciality_id FROM Speciality WHERE name=?", (speciality,))
            speciality_id = cursor.fetchone()[0]

            # Check if the (restaurant_id, speciality_id) combination already exists
            cursor.execute("""
                SELECT 1 FROM Restaurant_Speciality
                WHERE restaurant_id=? AND speciality_id=?
            """, (restaurant_id, speciality_id))
            exists = cursor.fetchone()

            if not exists:
                # Insert into Restaurant_Speciality table
                cursor.execute("""
                    INSERT INTO Restaurant_Speciality (restaurant_id, speciality_id)
                    VALUES (?, ?)
                """, (restaurant_id, speciality_id))

    conn.commit()
    conn.close()

populate_restaurant_speciality_table(df)


In [13]:
def populate_restaurant_social_media_table(df):
    conn = sqlite3.connect('restaurant.db')
    cursor = conn.cursor()

    for index, row in df.iterrows():
        if pd.notna(row['Social media / website']):
            social_media_links = row['Social media / website'].split(',')
            social_media_links = [link.strip() for link in social_media_links if link.strip()]

            for link in social_media_links:
                # Get restaurant_id
                cursor.execute("SELECT restaurant_id FROM Restaurant_Info WHERE name=?", (row['Name'],))
                restaurant_id = cursor.fetchone()[0]

                # Check if the (restaurant_id, url) combination already exists
                cursor.execute("""
                    SELECT 1 FROM Restaurant_Social_Media
                    WHERE restaurant_id=? AND url=?
                """, (restaurant_id, link))
                exists = cursor.fetchone()

                if not exists:
                    # Insert into Restaurant_Social_Media table
                    cursor.execute("""
                        INSERT INTO Restaurant_Social_Media (restaurant_id, url)
                        VALUES (?, ?)
                    """, (restaurant_id, link))

    conn.commit()
    conn.close()

populate_restaurant_social_media_table(df)
